In [1]:
# from facenet_pytorch import MTCNN, InceptionResnetV1
import sys
import os
sys.path.append("..")

from models.mtcnn import MTCNN
from models.inception_resnet_v1 import InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import numpy as np
import pandas as pd
import cv2
from PIL import Image, ImageDraw
from IPython.display import display

from models.utils.detect_face import save_img

workers = 0 if os.name == 'nt' else 4

#### Check Device

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


## Preprocessing

#### Define MTCNN

In [3]:
mtcnn = MTCNN(
    image_size=160, margin=25, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    keep_all=True, device=device
)

#### Define a dataset and data loader

In [4]:
def collate_fn(x):
    return x[0]

# dataset = datasets.ImageFolder('../data/toy')
dataset = datasets.ImageFolder('../data/img')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

#### Get all face images using by MTCNN

In [5]:
i = 0
for img, y in loader:
    _, x = mtcnn(img.convert('RGB'))
    for face in x:
        # display(face)
        # save_img(face, f'../data/new_toy/{i}.png')
        save_img(face, f'../data/cropped_img/{i}.png')
        i += 1
        
print('\nDone')


Done


### Preprocessing 결과
1. Raw data로 총 1022장의 사진을 사용  
2. MTCNN을 이용해 raw data로부터 2331개의 얼굴 후보를 추출  
3. 이후 직접 labeling을 통해 얼굴이 아닌 것들과 finetuning에 적합하지 않은 데이터를 분류를 통해 최종적으로 1225개의 얼굴 최종 데이터를 수집